In [69]:
import os
from typing import Optional
import requests
from langchain.tools import StructuredTool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage
from dotenv import load_dotenv
import datetime

In [70]:
load_dotenv()

True

In [71]:
def get_physicochemical_report(report_id: str) -> dict:
    base_url = os.getenv("PHYSICOCHEMICAL_API")
    if not base_url:
        raise ValueError(f"La variable de entorno '{base_url}' no está definida.")
    if not report_id.strip():
        return "Entrada vacía. Por favor, proporcione el ID de un reporte."
    report_id = report_id.strip().strip('"').strip("'").upper()
    try:
        url = f"{base_url}/{report_id}/reporte"
        response = requests.get(url)
        response.raise_for_status()
        return {
            "report_id": report_id,
            "url": url,
        }
    except requests.exceptions.RequestException as e:
        raise ValueError(
            f"No se pudo encontrar el reporte fisicoquímico con el código {report_id}. Error: {e}"
        )
    except ValueError as e:
        return str(e)

In [72]:
def get_document(titulo: Optional[str] = None, año: Optional[int] = None) -> dict:
    base_url = os.getenv("DOCUMENT_API")
    if not base_url:
        raise ValueError(f"La variable de entorno '{base_url}' no está definida.")
    
    params = {}
    if titulo:
        titulo = titulo.strip().strip('"').strip("'")
    params = {"search": titulo} if titulo else {}
    
    if año:
        params["search"] = año

    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status()
        data = response.json()
        documentos = data.get("values", [])
        return {
            "resultados": documentos,
        }
    except requests.exceptions.RequestException as e:
        return f"Error al buscar documentos: {e}"

In [73]:
def get_hydrobiological_report(report_id: Optional[str] = None) -> str:
    base_url = os.getenv("HYDROBIOLOGICAL_API")
    if not base_url:
        raise ValueError(f"La variable de entorno '{base_url}' no está definida.")
    if not report_id:
        return "Entrada vacía. Por favor, proporcione el nombre de un reporte."
    report_id = report_id.strip().strip('"').strip("'").upper()
    url = f"{base_url}/{report_id}/reporte"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return {
            "report_id": report_id,
            "url": url,
        }

    except requests.exceptions.RequestException as e:
        raise ValueError(
            f"No se pudo encontrar el reporte hidrobiológico con el código {report_id}."
        )

In [74]:
def get_todays_date() -> str:
    today = datetime.datetime.now()
    day_name = today.strftime("%A")
    date_string = today.strftime("%Y-%m-%d")
    return f"Hoy es {day_name}, {date_string}"

In [75]:
get_document_tool = StructuredTool.from_function(
    func=get_document,
    name="buscar_documentos_func",
    description="Busca documentos por título y opcionalmente por año. Ejemplo: Calidad del aire, año 2023.",
)
    
get_physicochemical_report_tool = StructuredTool.from_function(
    func=get_physicochemical_report,
    name="descargar_reporte_fisicoquimicos",
    description="Devuelve el enlace de descarga del reporte físico-químico dado el nombre.",
)

get_hydrobiological_report_tool = StructuredTool.from_function(
    func=get_hydrobiological_report,
    name="descargar_reporte_hidrobiologico",
    description="Devuelve el enlace de descarga del reporte hidrobiológico desde la API de Piragua usando un código.",
)

get_todays_date_tool = StructuredTool.from_function(
    func=get_todays_date,
    name="obtener_fecha_hoy",
    description="Devuelve la fecha y el día actual para tener como base para calculos de tiempo.",
)

tools = [
    get_document_tool,
    get_physicochemical_report_tool,
    get_hydrobiological_report_tool,
    get_todays_date_tool,
]

In [76]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    api_key=os.getenv("GENAI_API_KEY"),
    temperature=0.3,
)

llm_with_tools = llm.bind_tools(tools)

tool_list = {
    "descargar_reporte_hidrobiologico": get_hydrobiological_report_tool,
    "buscar_documentos_func": get_document_tool,
    "descargar_reporte_fisicoquimicos": get_physicochemical_report_tool,
    "obtener_fecha_hoy": get_todays_date_tool,
}

system_message = SystemMessage(
    content="Eres un asistente útil que siempre responde en español, pero nunca le dice al usuario que funciones se ejecutan."
)


In [87]:
# --- Ejecutar consulta ---
# query = "¿que documentos de calidad de aire se tienen reportados?"
# query = "Quiero el informe PDF de hidrobiología de código AMFQ01"
# query = "reporte fisicoquímico AMFQ01"
# query = "reporte fisicoquímico AMFQ01 y informe el informe hidrobiología AMFQ01"
# query = "hola como estas ? retorna el link del documento con el título Calidad del aire: reporte semanal del 06 al 12 de mayo de 2024"
# query = "¿Qué día es hoy?"
# query = "hola como estas ? retorna los nombres de los documentos año 2023 de Calidad del aire"
# query = "hola como estas ? retorna los nombres de los documentos solo de hace 4 años de Calidad del aire"
query = "¿puedes listar los documentos de calidad de aire se tienen reportados?"

# try:
#     today_str = get_todays_date()
#     messages = [
#         system_message,
#         ToolMessage(content=today_str, tool_call_id="fecha_actual"), 
#         HumanMessage(content=query)]
#     ai_messages = llm_with_tools.invoke(messages)
#     print(ai_messages)
#     messages.append(ai_messages)

#     for tool_call in ai_messages.tool_calls:
#         selected_tool = tool_list[tool_call["name"].lower()]
#         tool_output = selected_tool.invoke(tool_call["args"])
#         messages.append(ToolMessage(content=tool_output, tool_call_id=tool_call["id"]))

#     second_ai_message = llm_with_tools.invoke(messages)
#     print(llm_with_tools.invoke(messages).content)
# except Exception as e:
#     print(f"Error al ejecutar el agente: {str(e)}")

try:
    messages = [system_message, HumanMessage(content=query)]

    for _ in range(3):  # máximo 3 ciclos de invocación
        ai_message = llm_with_tools.invoke(messages)
        print(ai_message)
        messages.append(ai_message)

        if not ai_message.tool_calls:
            # Si el modelo ya generó respuesta final, la imprimimos
            print(ai_message.content)
            break

        # Ejecutamos cada herramienta invocada
        for tool_call in ai_message.tool_calls:
            tool_name = tool_call["name"].lower()
            tool_args = tool_call["args"]
            tool_id = tool_call["id"]

            selected_tool = tool_list.get(tool_name)
            if selected_tool:
                tool_output = selected_tool.invoke(tool_args)
                messages.append(ToolMessage(content=tool_output, tool_call_id=tool_id))
            else:
                print(f"⚠️ Herramienta no encontrada: {tool_name}")

    else:
        print("⚠️ Se alcanzó el límite de iteraciones sin obtener respuesta final.")

except Exception as e:
    print(f"Error al ejecutar el agente: {str(e)}")


content='' additional_kwargs={'function_call': {'name': 'buscar_documentos_func', 'arguments': '{"titulo": "Calidad del aire"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []} id='run-b031bcde-3a08-46aa-92e5-8ad4d1b014f0-0' tool_calls=[{'name': 'buscar_documentos_func', 'args': {'titulo': 'Calidad del aire'}, 'id': '089c8395-bf45-4dd9-93d4-1a1db900da3b', 'type': 'tool_call'}] usage_metadata={'input_tokens': 153, 'output_tokens': 10, 'total_tokens': 163, 'input_token_details': {'cache_read': 0}}
content='Se encontraron los siguientes reportes de calidad del aire:\n\n\n\n* Reporte semanal del 01 al 07 de enero de 2024\n\n* Reporte semanal del 15 al 21 de enero de 2024\n\n* Reporte semanal del 08 al 14 de enero de 2024\n\n* Reporte semanal del 22 al 28 de enero de 2024\n\n* Reporte semanal del 29 ene al 04 febrero de 2024\n\n* Reporte semanal del 05 al 11 de febrero de 202